## Load libraries


In [1]:
import pandas as pd
import numpy as np
import os
import random
import sklearn
from sklearn.impute import SimpleImputer
import seaborn as sb
from scipy import stats
from sklearn.ensemble import IsolationForest
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.datasets import dump_svmlight_file
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
X = pd.read_csv(os.path.join("data", "X_train.csv"),
                delimiter=',',
                index_col = 'id')
y = pd.read_csv(os.path.join("data", "y_train.csv"),
                delimiter=',',
                index_col='id')
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, SelectFromModel
from sklearn.ensemble import ExtraTreesRegressor
from scipy.stats import pearsonr



nan_idx = np.isnan(X)
imputer = KNNImputer(n_neighbors=20)
X_imp = imputer.fit_transform(X)

model = sklearn.ensemble.IsolationForest(max_samples=0.5, contamination=0.025)
# model = sklearn.neighbors.LocalOutlierFactor(contamination=0.025)
outl_pred = model.fit_predict(X_imp)
mask = outl_pred != -1


X_cleaned = np.copy(X_imp)
y_cleaned = np.copy(y.values)
X_cleaned, y_cleaned = X_cleaned[mask, :], y_cleaned[mask]
print(X_cleaned.shape, y_cleaned.shape)
y_train = y_cleaned


(1181, 832) (1181, 1)


In [2]:
thresh_corr = 0.1

df_train = pd.DataFrame(X_cleaned, columns=X.columns)
features = df_train.iloc[:,1:].columns.tolist()
labels = y_cleaned
corr = {}
for f in features:
    x_f = df_train[[f]][f].values
    key = f
    corr[key] = pearsonr(x_f,y_cleaned)[0]

data_corr = pd.DataFrame(corr, index=['lin']).T
selected_features = data_corr.loc[(abs(data_corr['lin']) >= thresh_corr) ].index
selected_features


/scratch_net/troubadour/tianfwang/conda_envs/pytorch3d/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/scratch_net/troubadour/tianfwang/conda_envs/pytorch3d/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/scratch_net/troubadour/tianfwang/conda_envs/pytorch3d/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/scratch_net/troubadour/tianfwang/conda_envs/pytorch3d/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not d

Index(['x2', 'x15', 'x21', 'x23', 'x26', 'x27', 'x29', 'x40', 'x69', 'x77',
       ...
       'x777', 'x778', 'x780', 'x788', 'x796', 'x801', 'x817', 'x819', 'x823',
       'x824'],
      dtype='object', length=192)

In [3]:
X_train = X_cleaned

X_train = sklearn.preprocessing.StandardScaler().fit(X_cleaned).transform(X_train)
X_train  = pd.DataFrame(X_train, columns=X.columns)
X_train = X_train[selected_features].values

In [ ]:
X_test = pd.read_csv(os.path.join("data", "X_test.csv"),
                         delimiter=',',
                         index_col='id')

X_test_KNN = imputer.fit(X_cleaned).transform(X_test)
X_test = pd.DataFrame(X_test_KNN, columns=X_test.columns, index=X_test.index)
# X_test = X_test[selected_features]
X_test_stdised = sklearn.preprocessing.StandardScaler().fit(X_cleaned).transform(X_test)


/scratch_net/troubadour/tianfwang/conda_envs/pytorch3d/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNNImputer was fitted without feature names
  warnings.warn(


In [ ]:
X_test_stdised = pd.DataFrame(X_test_stdised, columns=X_test.columns, index=X_test.index)
X_test_stdised = X_test_stdised[selected_features].values

In [ ]:
def r2_sc(preds, train_data):
    labels = train_data.get_label()
    return 'r2', r2_score(labels, preds), True

best = 0
trial_itr = 100
for i in range(trial_itr):
    print(f"train itr {i}")
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size = 0.15) 
    
    lgb_train = lgb.Dataset(X_train_split, y_train_split)
    
    
    lgb_eval = lgb.Dataset(X_val_split, y_val_split, reference=lgb_train)


    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 32,
        'learning_rate': 0.05,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
    }



    model = lgb.train(params,
                    lgb_train,
                    num_boost_round=1000,
                    feval=r2_sc,
                    valid_sets={lgb_train, lgb_eval},
                     early_stopping_rounds=100)
    y_val_predict = model.predict(X_val_split, num_iteration=model.best_iteration)
        
    
    
    res = r2_score(y_val_split, y_val_predict)
    
    print(res)
    if res > best:
        print(f"New Best result {res}")
        best = res
        y_pred = model.predict(X_test_stdised, num_iteration=model.best_iteration)
        f = open(f"out{best}.csv", "w")
        f.write("id,y\n")
        for i,x in enumerate(y_pred):
            f.write("{},{}\n".format(i,x))
        f.close()

In [9]:
import glob
print(glob.glob("./test/*"))

['./test/corr_linear.csv', './test/lgb_multirun.csv', './test/lgb.csv']


In [23]:
inputs = glob.glob("./ensemble/*")
print(inputs)
res = [0.0]*776

for f in inputs:
    f = open(f).readlines()[1:]
    for l in f:
        sp = l.split(',')
        idx, s = int(sp[0]), float(sp[1])
        res[idx] += s
out = open("./ensemble_debug.csv", "w")
out.write("id,y\n")
for i in range(776):
    out.write("{},{}\n".format(i, (res[i]/len(inputs))))
out.close()

['./test/corr_linear.csv', './test/lgb_multirun.csv', './test/lgb.csv']
